esm based dynamic model (not using static embeds).


Use TF:
*  🇰https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=de8419b5
* Torch based /Trainer example:  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb#scrollTo=49dcba23



* NOte for pytorch training could use trainer maybe, and mixed precision - https://huggingface.co/docs/transformers/v4.18.0/en/performance#fp16-training

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# # drive.mount("/content/drive/MyDrive/Research/proteins")
# # drive.mount("/content/drive/MyDrive/proteins")

# !pip install tensorflow -U -q
# !pip install torch accelerate transformers fair-esm datasets  -U -q


* Use the unirpot fasta file I downloaded and uploaded to my drive

`/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta`

* Download fasta from: `https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:0.9`

In [2]:
# DATA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/membrane_100_whole_protbert_embed.snappy.parquet"
# DATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/swp_human_viri_all_embed_esm.parquet" ## ESM1B embedding (max len 1022)
# DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-train.csv.gz"## TRAIN
DATA_PATH = "hum_vir_swp-train.csv.gz"## TRAIN

## TEST data:
# TEST_DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-test.csv.gz"## TRAIN
TEST_DATA_PATH = "hum_vir_swp-test.csv.gz"## TEST

# ## metadata for all reviewed/swissprot human + virus proteins
# METADATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/SWP_human_viruses_all.xlsx"

TARGET_COL = "virus" ## use for filtering data into 1 class

MAX_LEN = 512 # exclude sequences longer than this. (Not merely truncate)

In [3]:
# MY_FASTA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta" # 14k sequences, with fragments, 90% id redundnancy

# MY_FASTA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_whole_100.fasta" # 30K sequences, without fragments
# MY_FASTA_PATH = "Transmembrane_human_whole_90.fasta"

# Embed sequences in a FASTA file

In [4]:
# from bio_embeddings.embed import ProtTransBertBFDEmbedder
# from Bio import SeqIO
import torch
import esm
import tensorflow
import tensorflow as tf
from tensorflow.keras.metrics import AUC as tf_auc

import pandas as pd
import numpy as np
import random
# from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
# from sklearn.svm import OneClassSVM
from sklearn.preprocessing  import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm.autonotebook import tqdm

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

from transformers import TrainingArguments, Trainer, logging
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader
## https://huggingface.co/docs/transformers/perf_train_gpu_one

2023-08-29 12:43:55.211292: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 12:43:55.291922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 12:43:55.918048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_17514/3407002226.py:22: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
def auc(y_true, y_pred):
    auc = tensorflow.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [6]:
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)
# tensorflow.keras.mixed_precision.set_global_policy('mixed_float16') # causes error with our pretrained model

In [7]:
# embedder = ProtTransBertBFDEmbedder(half_model=True)

In [8]:
# df = pd.read_parquet(DATA_PATH) # numpy to pandas
df = pd.read_csv(DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
df_test = pd.read_csv(TEST_DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
 ## lengths of all the seqs
df

,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,Cluster: DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Cluster: Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Cluster: Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Cluster: Protein FAM9B
4,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...,0,1360,Cluster: TRAF2 and NCK-interacting protein kinase
...,...,...,...,...
20335,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...,0,578,Cluster: Dystrotelin
20336,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...,0,348,Cluster: Cell surface glycoprotein CD200 recep...
20337,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...,0,421,Cluster: Putative neuroblastoma breakpoint fam...
20338,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...,1,376,Cluster: Thymidine kinase


In [9]:
df.groupby(["virus"])["Length"].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
virus,,,,,,,,
0,14668.0,473.16,308.52,11.0,244.0,398.0,623.0,1533.0
1,5672.0,380.31,298.35,11.0,151.0,297.0,514.0,1520.0


In [10]:
print(df["virus"].agg(["mean","sum","count"]).round(2))
# df["Length"].describe().round(1)

mean         0.28
sum       5672.00
count    20340.00
Name: virus, dtype: float64


In [11]:
df_test["Length"].describe().round(1)

count    4777.0
mean      444.3
std       304.5
min        18.0
25%       212.0
50%       369.0
75%       589.0
max      1534.0
Name: Length, dtype: float64

In [12]:
df = df.loc[df["Length"]<=MAX_LEN].reset_index(drop=True)
df_test = df_test.loc[df_test["Length"]<=MAX_LEN].reset_index(drop=True)

print(df["Length"].describe().round(1))
print(df["virus"].agg(["mean","sum","count"]).round(2))

count    13727.0
mean       272.9
std        128.4
min         11.0
25%        161.0
50%        269.0
75%        376.0
max        512.0
Name: Length, dtype: float64
mean         0.31
sum       4247.00
count    13727.00
Name: virus, dtype: float64


In [13]:
# ## metadata about all sequences, can be used to identify and to define targets/labels
# df_meta = pd.read_excel(METADATA_PATH).dropna(how="all",axis=1)
# df_meta

### example pretrained fb/torch:
* https://github.com/facebookresearch/esm#getting-started-with-this-repo-

* Transformers + trainer example : (mlm case): https://github.com/facebookresearch/esm/discussions/556
* keras models supported / via HF?
  * https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example


  TF finetuning example (sequence evel?):
  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=4b26b828

In [19]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
import tensorflow
# from tensorflow.keras.metrics.AUC()
# from transformers import AutoTokenizer #DataCollatorForLanguageModeling,
## https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer, TFAutoModel
from transformers import TFAutoModelForTokenClassification, TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification

In [15]:
# sequences = df["Sequence"].tolist()
# labels = df["virus"].tolist()
# groups = df["Cluster name"].tolist()

# assert len(sequences) == len(labels) # Quick check to make sure we got it right

# train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)

In [16]:
train_sequences = df["Sequence"].tolist()
train_labels = df["virus"].tolist()
train_groups = df["Cluster name"].tolist()

# train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)

test_sequences = df_test["Sequence"].tolist()
test_labels = df_test["virus"].tolist()

In [20]:
# # Load ESM-2 model
# ## smallest: esm2_t6_8M_UR50D
# ## 2d smallest
# ## large: esm2_t33_650M_UR50D
# # model_checkpoint =  "facebook/esm2_t12_35M_UR50D"#"facebook/esm2_t6_8M_UR50D"
# model_checkpoint =  "facebook/esm2_t30_150M_UR50D"
#  # ElnaggarLab/ankh-base
#  ### https://github.com/agemagician/Ankh/blob/main/examples/binary_classification_solubility_task.ipynb - different model?
#  #  https://github.com/agemagician/Ankh#models   - 450M model size
# # model_checkpoint =   "ElnaggarLab/ankh-base"

# model_max_len = min(800,MAX_LEN)

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding=True, truncation=True,max_length=model_max_len)

# num_labels = max(train_labels + test_labels) #+ 1  # Add 1 since 0 can be a label
# print("Num labels:", num_labels)
# model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification") # worked, orig
# # model = EsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# # model = TFEsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification")

# # model.eval()  # disables dropout for deterministic results # from original - in infer/embed only example

# # last_layer_num = model.num_layers ## 33 for esm2_t33_650M_UR50D


In [21]:
## from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model


# Load pre-trained model and tokenizer
model_checkpoint = "facebook/esm2_t30_150M_UR50D"#"facebook/esm2_t6_8M_UR50D"  # Replace with your pre-trained model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Parameters
model_max_len = MAX_LEN
batch_size_train = 5
batch_size_test = 5
learning_rate = 8e-5
num_labels = 1  # Replace with the number of unique labels in your dataset

# Tokenize the sequences
train_tokenized = tokenizer(train_sequences, padding=True, truncation=True, max_length=model_max_len)
test_tokenized = tokenizer(test_sequences, padding=True, truncation=True, max_length=model_max_len)

# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokenized),
    train_labels
)).batch(batch_size_train).shuffle(buffer_size=len(train_sequences))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_tokenized),
    test_labels
)).batch(batch_size_test)

## added. (Awhat about add col labels?)
train_dataset = train_dataset.map(lambda x, y: ({'input_layer': x['input_ids'], 'attention_mask': x['attention_mask']}, y))
test_dataset = test_dataset.map(lambda x, y: ({'input_layer': x['input_ids'], 'attention_mask': x['attention_mask']}, y))

# Load the pre-trained transformer model
base_model = TFAutoModel.from_pretrained(model_checkpoint)

# Freeze all layers except the last few
for layer in base_model.layers[2:-4]:
    layer.trainable = False

# Create new model on top
input_layer = Input(shape=(model_max_len,), dtype=tf.int32, name='input_layer')
sequence_output = base_model(input_layer)[0]

# Add GlobalMeanPooling1D and GlobalMaxPooling1D
avg_pool = GlobalAveragePooling1D()(sequence_output)
max_pool = GlobalMaxPooling1D()(sequence_output)
concat = Concatenate()([avg_pool, max_pool])
concat = Dropout(0.15)(concat)

# # Initialize mixed precision training
# from tensorflow.keras import mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)


# Add output layer for binary classification
output = Dense(1, activation='sigmoid', dtype='float32')(concat)  # Ensuring the dtype is float32 for the output layer

# Compile model
final_model = Model(inputs=input_layer, outputs=output)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = "BinaryFocalCrossentropy"#tf.keras.losses.BinaryCrossentropy(from_logits=False)

final_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy",tf.keras.metrics.AUC(num_thresholds=10,from_logits=True)])

# Train the model
final_model.fit(train_dataset, validation_data=test_dataset, epochs=6)


2023-08-29 12:45:28.565627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFEsmModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'esm.embeddings.position_embeddings.weight', 'esm.embeddings.position_ids', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing TFEsmModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFEsmModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFEsmModel were not initialized from th

Epoch 1/5


2023-08-29 12:45:33.630760: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [13727,512]
	 [[{{node Placeholder/_1}}]]
2023-08-29 12:45:33.630902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32 and shape [13727,512]
	 [[{{node Placeholder/_0}}]]
/home/ddofer/anaconda3/lib/python3.10/site-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['attention_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


2023-08-29 12:45:58.348080: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xea09b360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-29 12:45:58.348103: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2023-08-29 12:45:58.351333: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-29 12:45:58.469214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-08-29 12:45:58.537726: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-08-29 12:45:58.582138: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2746/2746 [==============================] - ETA: 0s - loss: 0.0706 - accuracy: 0.8823 - auc: 0.8838

2023-08-29 13:00:24.084160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int32 and shape [3231]
	 [[{{node Placeholder/_2}}]]


2746/2746 [==============================] - 986s 343ms/step - loss: 0.0706 - accuracy: 0.8823 - auc: 0.8838 - val_loss: 0.0491 - val_accuracy: 0.9226 - val_auc: 0.9233
Epoch 2/5
2746/2746 [==============================] - 940s 342ms/step - loss: 0.0395 - accuracy: 0.9415 - auc: 0.9596 - val_loss: 0.0413 - val_accuracy: 0.9375 - val_auc: 0.9654
Epoch 3/5
2746/2746 [==============================] - 945s 344ms/step - loss: 0.0248 - accuracy: 0.9666 - auc: 0.9814 - val_loss: 0.0464 - val_accuracy: 0.9409 - val_auc: 0.9634
Epoch 4/5
2746/2746 [==============================] - 945s 344ms/step - loss: 0.0150 - accuracy: 0.9804 - auc: 0.9920 - val_loss: 0.0508 - val_accuracy: 0.9328 - val_auc: 0.9592
Epoch 5/5
2746/2746 [==============================] - 940s 342ms/step - loss: 0.0115 - accuracy: 0.9850 - auc: 0.9941 - val_loss: 0.0463 - val_accuracy: 0.9387 - val_auc: 0.9657


With 35M model :
```
Default (1024) max length
batch_size=8

opt = Adafactor(1e-4)##AdamWeightDecay(1e-4) #default: AdamWeightDecay(2e-5)
model.compile(optimizer=opt, metrics=["accuracy"],
              loss="BinaryCrossentropy")
3813/3813 [==============================] - 2625s 661ms/step - loss: 0.2452 - accuracy: 0.9104 - val_loss: 0.1622 - val_accuracy: 0.9363
Epoch 2/3
3813/3813 [==============================] - 2518s 661ms/step - loss: 0.1218 - accuracy: 0.9597 - val_loss: 0.1533 - val_accuracy: 0.9463
Epoch 3/3
3813/3813 [==============================] - 2523s 662ms/step - loss: 0.0730 - accuracy: 0.9799 - val_loss: 0.1978 - val_accuracy: 0.9436

```

In [22]:
model = final_model

In [23]:
# model.save("trained_esm.keras")
model.save("trained_esm_pool.tf",save_format="tf")

2023-08-29 14:04:56.443629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2023-08-29 14:04:59.482038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2023-08-29 14:04:59.714421: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_ids_1' with dtype int32 and shape [?,?]
	 [[{{n

2023-08-29 14:05:21.368006: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2023-08-29 14:05:21.613915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_ids_attention_mask' with dtype int32 and shape [?,?]
	 [[{{node input_ids_attention_mask}}]]
2023-08-29 14:05:21.850050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_ids_attention_mask' with dt

INFO:tensorflow:Assets written to: trained_esm_pool.tf/assets


INFO:tensorflow:Assets written to: trained_esm_pool.tf/assets


In [24]:
model.eval()

AttributeError: 'Functional' object has no attribute 'eval'

In [ ]:
y_test_pred = model.predict(tf_test_set)

In [ ]:
# print(classification_report(test_labels,y_test_pred.logits[:,1]>=0.5))
print(classification_report(test_labels,y_test_pred.logits>=0.5))

In [ ]:
roc_auc_score(test_labels,y_test_pred.logits)

In [ ]:
df_test.shape[0]

In [ ]:
y_test_binary_preds = (y_test_pred.logits>=0.5).astype(int).ravel()
print(len(y_test_binary_preds))

In [ ]:
test_mistakes_mask = test_labels!=y_test_binary_preds
print(test_mistakes_mask.sum())
print(len(test_mistakes_mask))

In [ ]:
df_test[test_mistakes_mask].drop_duplicates(subset="Cluster name")